In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn import preprocessing
from sklearn import metrics

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split


In [9]:
df = pd.read_csv('../../data/BitcoinHeistData.csv')
df = df.sample(frac=1)
df.head()


,address,year,day,length,weight,count,looped,neighbors,income,label
2648063,1Lbvw5MVZhvKMp33MfK1VnysxUztJmJP7u,2018,62,144,9.841741e-02,4639,0,2,4.277130e+07,white
456738,186eCZsMQobm92K2zcisRVA8gCxPuTujDg,2012,61,40,4.768372e-07,1,0,2,5.280078e+10,white
1383514,1CEQGvg5Z6dydSVMJTVu1wdzZmapDHT6BQ,2014,257,144,2.580301e-01,1281,0,1,9.994000e+08,white
235094,1C6VxEJ3HNCFo3X3ti4w9wXkQA3mLhVvtB,2011,204,16,1.000000e+00,1,0,2,4.480000e+08,white
2287097,1BU27Ud9qyUnegJRgvtSpRsDZqiRMCn8dZ,2017,66,0,1.000000e+00,1,0,1,7.902508e+07,white


In [3]:
df.isnull().sum()


address      0
year         0
day          0
length       0
weight       0
count        0
looped       0
neighbors    0
income       0
label        0
dtype: int64

In [5]:
def split(df):
    df = df.drop(['address'], axis=1)
    X = df.drop('label', axis=1)
    y = df['label']
    X_train, X_, y_train, y_ = train_test_split(X, y, test_size=0.3)
    X_val, X_test, y_val, y_test = train_test_split(X_, y_, test_size=0.5)
    return X_train, X_val, X_test, y_train, y_val, y_test

X_train, X_val, X_test, y_train, y_val, y_test = split(df)

In [8]:
def dt():
    for d in [4, 8, 10, 15, 20]:
        clf = DecisionTreeClassifier(criterion='gini', max_depth=d)
        clf.fit(X_train, y_train)
        print(f'gini, max-depth:{d}, score:{clf.score(X_test, y_test)}')

    for d in [4, 8, 10, 15, 20]:
        clf = DecisionTreeClassifier(criterion='entropy', max_depth=d)
        clf.fit(X_train, y_train)
        print(f'entropy, max-depth:{d}, score:{clf.score(X_test, y_test)}')


dt()

gini, max-depth:4, score:0.9857921623752871
gini, max-depth:8, score:0.9863612987280146
gini, max-depth:10, score:0.9868892926937978
gini, max-depth:15, score:0.9879247094318923
gini, max-depth:20, score:0.9866195814904972
entropy, max-depth:4, score:0.9857921623752871
entropy, max-depth:8, score:0.986091587524714
entropy, max-depth:10, score:0.9872435743591502
entropy, max-depth:15, score:0.9880847076033417
entropy, max-depth:20, score:0.986285871018617
